# 02 Correct Network Errors with Project Cards

In [1]:
import os
import sys
import yaml
import pickle
import glob

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

## Remote I/O and Parameters

In [4]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
root_dir = os.path.join('C:\\' 'Users', 'david.ory', 'Documents', 'GitHub', 'travel-model-two-networks')
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_02')
output_dir = input_dir
card_dir = os.path.join(root_dir, 'project_cards')
# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')
lasso_dir = os.path.join('C:\\' 'Users', 'david.ory', 'Documents', 'GitHub', 'mtc_Lasso')

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2020-11-03 10:00:29, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\mtc_Lasso
2020-11-03 10:00:29, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\mtc_Lasso


## Read Version 00 Scenario

In [6]:
version_00_pickle_file_name = os.path.join(input_dir, 'working_scenario_00.pickle')
v_00_scenario = pickle.load(open(version_00_pickle_file_name, 'rb'))

In [7]:
v_00_scenario.road_net.links_df.shape

(1634769, 31)

## Create Scenario and Apply Project Cards

In [8]:
v_01_scenario = Scenario.create_scenario(
    base_scenario=v_00_scenario,
    card_directory = card_dir,
    validate_project_cards=False)

2020-11-03 10:01:14, INFO: Creating Scenario
2020-11-03 10:01:14, INFO: Creating Scenario


In [9]:
v_01_scenario.apply_all_projects()

2020-11-03 10:01:16, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2020-11-03 10:01:16, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2020-11-03 10:01:16, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2020-11-03 10:01:16, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2020-11-03 10:01:16, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2020-11-03 10:01:16, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass


C:\Users\david.ory\anaconda3\envs\mtc_lasso\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2020-11-03 10:01:17, INFO: Applying Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2020-11-03 10:01:17, INFO: Applying Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2020-11-03 10:01:17, INFO: Applying Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2020-11-03 10:01:17, INFO: Applying Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2020-11-03 10:01:17, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2020-11-03 10:01:17, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass


C:\Users\david.ory\anaconda3\envs\mtc_lasso\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2020-11-03 10:01:19, INFO: Applying Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma
2020-11-03 10:01:19, INFO: Applying Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma
2020-11-03 10:01:19, INFO: Applying Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma
2020-11-03 10:01:19, INFO: Applying Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma
2020-11-03 10:01:19, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma
2020-11-03 10:01:19, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma


...

C:\Users\david.ory\anaconda3\envs\mtc_lasso\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2020-11-03 10:05:49, INFO: Applying Year 2015 Exclude Trucks on SR 85
2020-11-03 10:05:49, INFO: Applying Year 2015 Exclude Trucks on SR 85
2020-11-03 10:05:49, INFO: Applying Year 2015 Exclude Trucks on SR 85
2020-11-03 10:05:49, INFO: Applying Year 2015 Exclude Trucks on SR 85
2020-11-03 10:05:49, INFO: Applying Project to Roadway Network: Year 2015 Exclude Trucks on SR 85
2020-11-03 10:05:49, INFO: Applying Project to Roadway Network: Year 2015 Exclude Trucks on SR 85
2020-11-03 10:05:50, INFO: Applying Year 2015 Exclude Trucks on I580
2020-11-03 10:05:50, INFO: Applying Year 2015 Exclude Trucks on I580
2020-11-03 10:05:50, INFO: Applying Year 2015 Exclude Trucks on I580
2020-11-03 10:05:50, INFO: Applying Year 2015 Exclude Trucks on I580
2020-11-03 10:05:50, INFO: Applying Project to Roadway Network: Year 2015 Exclude Trucks on I580
2020-11-03 10:05:50, INFO: Applying Project to Roadway Network: Year 2015 Exclude Trucks on I580


C:\Users\david.ory\anaconda3\envs\mtc_lasso\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2020-11-03 10:05:50, INFO: Applying Year 2015 Modify HOV Lanes-Fastrak Lanes and USE on Antioch Bridge NB on Route 160 - Bridge Pass
2020-11-03 10:05:50, INFO: Applying Year 2015 Modify HOV Lanes-Fastrak Lanes and USE on Antioch Bridge NB on Route 160 - Bridge Pass
2020-11-03 10:05:50, INFO: Applying Year 2015 Modify HOV Lanes-Fastrak Lanes and USE on Antioch Bridge NB on Route 160 - Bridge Pass
2020-11-03 10:05:50, INFO: Applying Year 2015 Modify HOV Lanes-Fastrak Lanes and USE on Antioch Bridge NB on Route 160 - Bridge Pass


C:\Users\david.ory\anaconda3\envs\mtc_lasso\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [10]:
v_01_scenario.applied_projects

['Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass',
 'Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass',
 'Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma',
 'Year 2015 Modify USE and HOV Lanes on US-101 SB (Segment 02) from South of Healdsburg to North of Petaluma',
 'Year 2015 Modify HOV Lanes and USE on US 101 SB CA-85 to Oregon Expressway',
 'Year 2015 Modify HOV Lanes and USE on US 101 SB CA-85 to Cochrane Road',
 'Year 2015 Modify HOV Lanes and USE on US-101 NB (Segment 02) from North of Petaluma to South of Healdsburg',
 'Year 2015 Modify HOV Lanes and USE on US 101 NB Oregon Expressway to Whipple Road',
 'Year 2015 Modify HOV Lanes and USE on US-101 NB from North of Petaluma to South of Healdsburg',
 'Year 2015 Modify HOV Lanes and USE on US 101 NB Cochran Road to CA-85',
 'Year 2015 Modify HOV Lanes and USE on San Tomas Expressway SB I-8

In [11]:
v_01_scenario.road_net.links_df.columns

Index(['access', 'bike_access', 'drive_access', 'fromIntersectionId', 'lanes',
       'maxspeed', 'name', 'oneWay', 'ref', 'roadway', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'u', 'v', 'walk_access', 'wayId',
       'county', 'model_link_id', 'A', 'B', 'rail_traveltime', 'rail_only',
       'locationReferences', 'shape_id', 'geometry', 'ft_cal', 'ft',
       'useclass', 'assignable', 'transit', 'managed', 'ML_lanes',
       'segment_id', 'tollbooth', 'ML_useclass', 'ML_access', 'ML_egress',
       'tollseg', 'tollboth', 'bus_only', 'transit_access'],
      dtype='object')

## Make Travel Model Networks

In [12]:
model_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = v_01_scenario.road_net, 
    parameters = parameters)

2020-11-03 10:12:08, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\mtc_Lasso
2020-11-03 10:12:08, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\mtc_Lasso


In [13]:
model_net = mtc.add_centroid_and_centroid_connector(
    roadway_network = model_net,
    parameters = parameters)

2020-11-03 10:12:08, INFO: Adding centroid and centroid connector to standard network
2020-11-03 10:12:08, INFO: Adding centroid and centroid connector to standard network
2020-11-03 10:12:08, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\mtc_Lasso
2020-11-03 10:12:08, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\mtc_Lasso
2020-11-03 10:12:20, INFO: Finished adding centroid and centroid connectors
2020-11-03 10:12:20, INFO: Finished adding centroid and centroid connectors


In [14]:
print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(2221035, 42)
(688952, 11)
(1163103, 7)


In [15]:
model_net = mtc.roadway_standard_to_mtc_network(model_net, parameters)

2020-11-03 10:12:21, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2020-11-03 10:12:21, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2020-11-03 10:12:21, INFO: Creating managed lane network.
2020-11-03 10:12:21, INFO: Creating managed lane network.
2020-11-03 10:12:21, INFO: Creating network with duplicated managed lanes
2020-11-03 10:12:21, INFO: Creating network with duplicated managed lanes


C:\Users\david.ory\anaconda3\envs\mtc_lasso\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
C:\Users\david.ory\anaconda3\envs\mtc_lasso\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
C:\Users\david.ory\anaconda3\envs\mtc_lasso\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

2020-11-03 10:27:10, INFO: Determining cntype
2020-11-03 10:27:10, INFO: Determining cntype
2020-11-03 10:27:10, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\mtc_Lasso
2020-11-03 10:27:10, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\mtc_Lasso
2020-11-03 10:27:10, INFO: Calculating and adding roadway network variable: cntype
2020-11-03 10:27:10, INFO: Calculating and adding roadway network variable: cntype
2020-11-03 10:28:22, INFO: Finished determining variable: cntype
2020-11-03 10:28:22, INFO: Finished determining variable: cntype
2020-11-03 10:28:22, INFO: Determining transit
2020-11-03 10:28:22, INFO: Determining transit
2020-11-03 10:28:22, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\mtc_Lasso
2020-11-03 10:28:22, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\mtc_Lasso
2020-11-03 10:28:22, INFO: Variable 'transit' updated for some links. Returning without overwriting for those 

## Write to Disk

## As shapefile

In [16]:
model_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
    link_output_variables = ["model_link_id", "A", "B", "geometry"],
    node_output_variables = ["model_node_id", "geometry"],
    data_to_csv = False,
    data_to_dbf = True,
)

2020-11-03 10:33:29, INFO: Writing Network as Shapefile
2020-11-03 10:33:29, INFO: Writing Network as Shapefile
2020-11-03 10:33:29, INFO: Renaming DBF Node Variables
2020-11-03 10:33:29, INFO: Renaming DBF Node Variables
2020-11-03 10:33:29, INFO: Renaming variables so that they are DBF-safe
2020-11-03 10:33:29, INFO: Renaming variables so that they are DBF-safe
2020-11-03 10:33:46, INFO: Renaming DBF Link Variables
2020-11-03 10:33:46, INFO: Renaming DBF Link Variables
2020-11-03 10:33:46, INFO: Renaming variables so that they are DBF-safe
2020-11-03 10:33:46, INFO: Renaming variables so that they are DBF-safe
2020-11-03 10:33:54, INFO: Writing Node Shapes:
 - C:\Users\david.ory\Documents\GitHub\travel-model-two-networks\data\processed\version_02\nodes.shp
2020-11-03 10:33:54, INFO: Writing Node Shapes:
 - C:\Users\david.ory\Documents\GitHub\travel-model-two-networks\data\processed\version_02\nodes.shp
2020-11-03 10:34:56, INFO: Writing Link Shapes:
 - C:\Users\david.ory\Documents\Gi

## As fixed width (for CUBE)

In [17]:
model_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
    #drive_only = True
)

2020-11-03 10:38:50, INFO: Starting fixed width convertion
2020-11-03 10:38:50, INFO: Starting fixed width convertion
2020-11-03 11:00:02, INFO: Writing out link database
2020-11-03 11:00:02, INFO: Writing out link database
2020-11-03 11:00:19, INFO: Writing out link header and width ----
2020-11-03 11:00:19, INFO: Writing out link header and width ----
2020-11-03 11:00:19, INFO: Starting fixed width convertion
2020-11-03 11:00:19, INFO: Starting fixed width convertion
2020-11-03 11:02:05, INFO: Writing out node database
2020-11-03 11:02:05, INFO: Writing out node database
2020-11-03 11:02:09, INFO: Writing out node header and width
2020-11-03 11:02:09, INFO: Writing out node header and width


In [18]:
%%capture
os.chdir(output_dir)
!make_cube.bat

In [19]:
%%capture
!runtpp make_simple_roadway_network.s
!runtpp make_complete_marin_network.s
!runtpp make_simple_transit_network.s

## As transit line file (for CUBE)

In [20]:
standard_transit_net = StandardTransit.fromTransitNetwork(v_01_scenario.transit_net, parameters = parameters)
mtc.write_as_cube_lin(standard_transit_net, parameters, outpath = os.path.join(output_dir, "transit.lin"))

2020-11-03 11:03:35, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\mtc_Lasso
2020-11-03 11:03:35, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\mtc_Lasso
2020-11-03 11:03:35, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2020-11-03 11:03:35, INFO: Converting GTFS Standard Properties to MTC's Cube Standard


## As pickle

In [21]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_01.pickle')
pickle.dump(v_01_scenario, open(working_scenario_filename, 'wb'))